In [1]:

# ## 🚀 SFO 프로젝트 피처엔지니어링 -dtw_clustering

# - 원본 공통전처리 파일(baseline_processed.csv) 로드부터 `df_dtw` 생성
# - 생성된 df_dtw을 csv로 저장
#
# [시작 전 준비사항]
# 1. `baseline_processed.csv'  파일 경로 확인 (input_csv_file)
# 2. 필요한 라이브러리 설치: `pip install pandas numpy scikit-learn ipykernel `


In [2]:

import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from itertools import combinations

In [3]:

# --- 0. 설정 및 파일 경로 정의 ---
input_csv_file = '../../data/baseline_processed.csv'  
output_final_csv_file = '../../data/dtw_clustering.csv' 


date_column = 'Date'
product_column = 'Product_Number'
target_column = 'T일 예정 수주량'



print("\n--- [1단계: dtw_clustering 피처 엔지니어링] ---")

df_dtw = None

df_train = None
df_test = None
cutoff_date = None


try:
    if not os.path.exists(input_csv_file):
        raise FileNotFoundError(f"🚨 오류: 입력 파일 '{input_csv_file}'을(를) 찾을 수 없습니다. 경로를 확인해주세요.")

    df_raw = pd.read_csv(input_csv_file)
    print(f"1.1.공통 전처리 파일 로드 완료 (shape: {df_raw.shape})")


    # 필수 컬럼 존재 여부 확인
    required_cols = {date_column, product_column, target_column}
    missing_cols = required_cols - set(df_raw.columns)
    if missing_cols:
        raise ValueError(f"🚨 필수 컬럼 {missing_cols} 이(가) 없습니다. CSV 컬럼명을 확인하세요.")


    # 날짜 컬럼 datetime 변환
    df_raw[date_column] = pd.to_datetime(df_raw[date_column], errors='coerce')
    if df_raw[date_column].isna().any():
        bad_rows = df_raw[df_raw[date_column].isna()].head()
        raise ValueError(
            "🚨 날짜 파싱 실패한 행이 있습니다. 일부 예시:\n"
            + str(bad_rows)
        )



    # 5. 날짜/제품 기준 정렬 (시계열 안정성 확보)
    df_raw = df_raw.sort_values(by=[date_column, product_column]).reset_index(drop=True)

    # 6. cutoff_date 계산 (날짜 고유값 기준 80%)
    unique_dates = pd.Series(df_raw[date_column].dropna().sort_values().unique())
    total_days = len(unique_dates)

    if total_days == 0:
        raise ValueError("🚨 고유 날짜가 0입니다. 데이터셋이 비어있거나 Date 파싱이 잘못된 것 같습니다.")

    cut_index = int(total_days * 0.8) - 1  # 0-based index 보정
    if cut_index < 0:
        cut_index = 0
    if cut_index >= total_days:
        cut_index = total_days - 1

    cutoff_date = unique_dates.iloc[cut_index]

    print(f"1.2 전체 고유 일자 수: {total_days}")
    print(f"    80% 위치 인덱스: {cut_index}")
    print(f"    cutoff_date: {cutoff_date.date()} (이 날짜까지 train으로 사용)")

    # 7. train / test 분리
    df_train = df_raw[df_raw[date_column] <= cutoff_date].copy()
    df_test  = df_raw[df_raw[date_column] >  cutoff_date].copy()

    # 8. 누수 방지 점검: test에 train과 겹치는 날짜가 있으면 안 됨
    if not df_test.empty:
        if (df_test[date_column] <= df_train[date_column].max()).any():
            raise RuntimeError(
                "🚨 누수 의심: test에 train보다 과거나 같은 날짜가 포함되어 있습니다. split 로직을 확인하세요."
            )

    # 9. split 결과 로그 출력
    train_start = df_train[date_column].min().date() if not df_train.empty else None
    train_end   = df_train[date_column].max().date() if not df_train.empty else None
    test_start  = df_test[date_column].min().date()  if not df_test.empty else None
    test_end    = df_test[date_column].max().date()  if not df_test.empty else None

    print(f"1.3 train 기간: {train_start} ~ {train_end}  (rows: {len(df_train):,})")
    print(f"    test  기간: {test_start} ~ {test_end}  (rows: {len(df_test):,})")

    # 10. df_dtw 미리보기 (메모리 상에서만 확인)
    df_train_preview = df_train[[product_column, date_column, target_column]].copy()
    df_train_preview["__SPLIT__"] = "train"

    df_test_preview = df_test[[product_column, date_column, target_column]].copy()
    df_test_preview["__SPLIT__"] = "test"

    df_preview = pd.concat([df_train_preview.head(), df_test_preview.head()], ignore_index=True)

    print("\n1.4 split 결과 미리보기 (상위 일부):")
    print(df_preview)

    print("\n✅ 1단계 완료: df_train / df_test / cutoff_date 메모리에 준비되었습니다.")
    print("   - df_train: train 데이터프레임 (DTW 클러스터링에 사용할 구간)")
    print("   - df_test : 미래 데이터프레임 (cluster_id만 붙일 예정)")
    print("   - cutoff_date: train/test 경계 날짜")




    # df_dtw.to_csv(output_final_csv_file, index=False, encoding='utf-8-sig')
    # print(f"dtw_clustering 피처 엔지니어링 완료! (df_dtw shape: {df_dtw.shape})")

except FileNotFoundError as e:
    print(f"🚨 {e}")
    df_time = None
    df_train = None
    df_test = None
    cutoff_date = None

except Exception as e:
    print(f"🚨 dtw_clustering 피처 엔지니어링 중 오류 발생: {e}")
    df_time = None
    df_train = None
    df_test = None
    cutoff_date = None




--- [1단계: dtw_clustering 피처 엔지니어링] ---
1.1.공통 전처리 파일 로드 완료 (shape: (10624, 19))
1.2 전체 고유 일자 수: 95
    80% 위치 인덱스: 75
    cutoff_date: 2022-04-22 (이 날짜까지 train으로 사용)
1.3 train 기간: 2022-01-26 ~ 2022-04-22  (rows: 8,524)
    test  기간: 2022-04-23 ~ 2022-05-11  (rows: 2,100)

1.4 split 결과 미리보기 (상위 일부):
  Product_Number       Date  T일 예정 수주량 __SPLIT__
0     Product_84 2022-01-26          0     train
1     Product_85 2022-01-26          0     train
2     Product_86 2022-01-26        616     train
3     Product_87 2022-01-26        104     train
4     Product_88 2022-01-26        187     train
5     Product_84 2022-04-23          3      test
6     Product_85 2022-04-23         13      test
7     Product_86 2022-04-23        559      test
8     Product_87 2022-04-23         92      test
9     Product_88 2022-04-23        239      test

✅ 1단계 완료: df_train / df_test / cutoff_date 메모리에 준비되었습니다.
   - df_train: train 데이터프레임 (DTW 클러스터링에 사용할 구간)
   - df_test : 미래 데이터프레임 (cluster_id만 붙일 예정)
   - cuto

In [4]:
# --- 2단계: train에서 SKU별 주 단위 시계열 생성 & 정규화 준비 ---


print("\n--- [2단계: SKU별 주 단위 시계열 생성 & z-score 정규화 준비] ---")

# 필수 전역 변수들이 존재하는지 확인
needed_vars = ["df_train", "cutoff_date"]
for var_name in needed_vars:
    if var_name not in globals() or globals()[var_name] is None:
        raise RuntimeError(f"🚨 '{var_name}' 이(가) 정의되어 있지 않습니다. 1단계 셀을 먼저 실행해야 합니다.")

# 우리가 이후에 계속 참조할 컬럼명들 (1단계와 동일하게 유지)
date_column = 'Date'
product_column = 'Product_Number'
target_column = 'T일 예정 수주량'

# 2.1 train 데이터만 사용 (안전하게 copy)
work_df = df_train[[product_column, date_column, target_column]].copy()

# 2.2 혹시라도 null target이 있으면 0으로 간주할지 / 드롭할지 결정
#     여기서는 생산 계획 관점에서 "기록이 있지만 수주량이 0"과 "기록 자체가 없음"을 구분하고 싶으므로
#     우선은 dropna() 하지 않고, 결측은 0으로 채우는 보수적 전략은 나중 단계에서 판단.
#     우선은 결측이 있으면 경고만 띄우자.
if work_df[target_column].isna().any():
    null_count = work_df[target_column].isna().sum()
    print(f"⚠ 경고: train에서 '{target_column}'에 NaN {null_count}개 존재. NaN은 0으로 대체합니다.")
    work_df[target_column] = work_df[target_column].fillna(0)

# 2.3 각 SKU별로 주 단위 시계열 만들기
#     - resample('W')는 기본적으로 주 단위 끝(일요일 기준)로 집계
#     - 공급/생산 측에서는 "그 주에 총 얼마나 필요했나"가 중요한 신호라 sum 사용
sku_weekly_sum = {}

for sku, sku_df in work_df.groupby(product_column):
    # 개별 SKU만 뽑아서 시계열 정렬
    sku_df = sku_df.sort_values(by=date_column)

    # Date를 인덱스로 설정해서 주 단위 리샘플
    sku_series = (
        sku_df.set_index(date_column)[target_column]
        .resample('W')  # 주 단위
        .sum()          # 그 주의 총 예정 수주량
    )

    # 완전히 0만 있는 SKU도 일단 그대로 둠 (나중에 z-score에서 표준편차 0으로 처리)
    sku_weekly_sum[sku] = sku_series

print(f"2.4 SKU 개수 (train에서 관측된): {len(sku_weekly_sum)}")

# 2.4 관측 길이가 너무 짧은 SKU는 제외 후보로 표시
#     예: 관측 주차 수가 3주 미만이면 "패턴"이라고 부르기 어렵다고 가정
min_weeks_required = 3

valid_skus = []
lowdata_skus = []

for sku, series in sku_weekly_sum.items():
    # 유효 관측 주(= 값이 전부 0이어도 주 단위로 존재하면 count로 잡힘)
    observed_weeks = series.shape[0]
    if observed_weeks < min_weeks_required:
        lowdata_skus.append(sku)
    else:
        valid_skus.append(sku)

print(f"2.5 유효 SKU(주 단위 {min_weeks_required}주 이상): {len(valid_skus)}개")
print(f"    관측 적어서 제외된 SKU: {len(lowdata_skus)}개 (예: {lowdata_skus[:5]})")

# 2.6 z-score 정규화 (valid_skus만)
#     - 각 SKU 시계열을 자기 자신의 평균/표준편차로 정규화
#     - 표준편차가 0이면 전부 0으로 (완전 안정 SKU로 간주)
sku_weekly_z = {}

for sku in valid_skus:
    series = sku_weekly_sum[sku].astype(float)

    mean_val = series.mean()
    std_val = series.std(ddof=0)  # 모분산 기준 (ddof=0), 안정적으로 0 체크 가능

    if std_val == 0 or np.isclose(std_val, 0.0):
        z_series = pd.Series(
            np.zeros_like(series.values, dtype=float),
            index=series.index
        )
    else:
        z_series = (series - mean_val) / std_val

    sku_weekly_z[sku] = z_series

print(f"2.7 z-score 변환 완료 (유효 SKU {len(sku_weekly_z)}개)")

# 2.8 길이 표준화(패딩/트리밍)는 아직 안 함
#     지금은 각 SKU별로 주차 길이가 다를 수 있음
#     다음 단계(3단계)에서 DTW로 거리를 계산할 때, DTW는 길이가 달라도 괜찮으니까
#     길이 맞출 필요 없이 그대로 distance matrix 계산 가능.

# 2.9 요약 프린트
example_sku = valid_skus[0] if valid_skus else None
if example_sku is not None:
    print("\n[샘플 SKU 1개 weekly 시계열 (원본 합계)]")
    print(sku_weekly_sum[example_sku].head())

    print("\n[샘플 SKU 1개 weekly 시계열 (z-score 정규화)]")
    print(sku_weekly_z[example_sku].head())

print("\n✅ 2단계 완료:")
print(" - sku_weekly_sum: SKU별 주 단위 수요 합계 시계열 (train만 사용)")
print(" - sku_weekly_z:   DTW용으로 z-score 정규화된 시계열 (길이 달라도 OK)")
print(" - valid_skus:     DTW 클러스터링 후보 SKU 목록")
print(" - lowdata_skus:   관측 적어서 cluster_id = -1 후보")



--- [2단계: SKU별 주 단위 시계열 생성 & z-score 정규화 준비] ---
2.4 SKU 개수 (train에서 관측된): 117
2.5 유효 SKU(주 단위 3주 이상): 117개
    관측 적어서 제외된 SKU: 0개 (예: [])
2.7 z-score 변환 완료 (유효 SKU 117개)

[샘플 SKU 1개 weekly 시계열 (원본 합계)]
Date
2022-01-30     0
2022-02-06     0
2022-02-13     0
2022-02-20    12
2022-02-27     8
Freq: W-SUN, Name: T일 예정 수주량, dtype: int64

[샘플 SKU 1개 weekly 시계열 (z-score 정규화)]
Date
2022-01-30   -0.646788
2022-02-06   -0.646788
2022-02-13   -0.646788
2022-02-20    2.155959
2022-02-27    1.221710
Freq: W-SUN, Name: T일 예정 수주량, dtype: float64

✅ 2단계 완료:
 - sku_weekly_sum: SKU별 주 단위 수요 합계 시계열 (train만 사용)
 - sku_weekly_z:   DTW용으로 z-score 정규화된 시계열 (길이 달라도 OK)
 - valid_skus:     DTW 클러스터링 후보 SKU 목록
 - lowdata_skus:   관측 적어서 cluster_id = -1 후보


In [5]:
# --- 3단계: DTW 거리 행렬 계산 ---

print("\n--- [3단계: DTW 거리 행렬 계산] ---")

# 필수 전역 변수들이 있는지 체크
needed_vars = ["sku_weekly_z", "valid_skus"]
for var_name in needed_vars:
    if var_name not in globals() or globals()[var_name] is None:
        raise RuntimeError(f"🚨 '{var_name}' 이(가) 정의되어 있지 않습니다. 2단계 셀을 먼저 실행해야 합니다.")

# 3.1 DTW 함수 정의
#     - 표준 동적 계획법 구현 (O(len(a)*len(b)))
#     - 거리 = |a_i - b_j| 의 누적 최소 합 (L1 distance 기반)
def dtw_distance(seq_a: np.ndarray, seq_b: np.ndarray) -> float:
    """
    seq_a, seq_b: 1D numpy arrays (float)
    return: DTW alignment cost (float)
    """
    len_a = len(seq_a)
    len_b = len(seq_b)

    # DP 매트릭스 초기화: 매우 큰 값으로 채움
    dp = np.full((len_a + 1, len_b + 1), np.inf, dtype=float)
    dp[0, 0] = 0.0

    # 동적 계획: 삽입/삭제/대각선 이동 중 최소 비용 경로 누적
    for i in range(1, len_a + 1):
        ai = seq_a[i - 1]
        for j in range(1, len_b + 1):
            bj = seq_b[j - 1]
            cost = abs(ai - bj)

            dp[i, j] = cost + min(
                dp[i - 1, j],     # 삭제 (a에서 한 걸음)
                dp[i, j - 1],     # 삽입 (b에서 한 걸음)
                dp[i - 1, j - 1]  # 매칭 (대각선)
            )
    return float(dp[len_a, len_b])

# 3.2 sku_weekly_z에서 사용할 SKU 순서 고정
#     - valid_skus는 우리가 2단계에서 관측 기간 충분하다고 판정한 SKU 목록
sku_order = list(valid_skus)

if len(sku_order) == 0:
    raise RuntimeError("🚨 valid_skus가 비어 있습니다. 유효한 SKU가 없으면 클러스터링을 할 수 없습니다.")

print(f"3.2 클러스터링 대상 SKU 수: {len(sku_order)}")

# 3.3 각 SKU의 시계열을 numpy array로 변환해서 캐시
#     - 시계열 index(주차 날짜)는 DTW에서 직접 안 쓰고 값 벡터만 쓴다.
sku_series_cache = {}
for sku in sku_order:
    series_z = sku_weekly_z[sku]
    # series_z는 pandas Series (index=주별 날짜, values=z-score된 수요)
    sku_series_cache[sku] = series_z.to_numpy(dtype=float)

# 3.4 DTW 거리 행렬 계산
n = len(sku_order)
dist_matrix = np.zeros((n, n), dtype=float)

for i in range(n):
    seq_i = sku_series_cache[sku_order[i]]
    for j in range(i + 1, n):
        seq_j = sku_series_cache[sku_order[j]]

        d = dtw_distance(seq_i, seq_j)
        dist_matrix[i, j] = d
        dist_matrix[j, i] = d

# 대각선은 자기 자신이라 0 그대로 두면 됨

print("3.5 DTW 거리 행렬 계산 완료.")
print(f"    dist_matrix shape: {dist_matrix.shape}")
print("    dist_matrix[0:3, 0:3] 예시:")
print(dist_matrix[0:3, 0:3])

# 3.6 간단 sanity check:
#     - 대칭 여부
#     - 음수 거리 없는지
is_symmetric = np.allclose(dist_matrix, dist_matrix.T, atol=1e-12)
has_negative = (dist_matrix < 0).any()

print(f"3.6 대칭 여부: {is_symmetric}")
print(f"    음수 거리 존재?: {has_negative}")

if not is_symmetric:
    raise RuntimeError("🚨 dist_matrix가 대칭이 아닙니다. DTW 계산 루프를 확인하세요.")
if has_negative:
    raise RuntimeError("🚨 dist_matrix에 음수 거리가 있습니다. DTW distance 정의를 확인하세요.")

print("\n✅ 3단계 완료:")
print(" - sku_order: 거리행렬 인덱스와 SKU를 매핑할 리스트")
print(" - dist_matrix: SKU 간 DTW 거리행렬 (shape = [N_SKU, N_SKU])")
print("이제 이걸 가지고 k 후보별 클러스터링과 실루엣 스코어를 구할 수 있습니다 (다음 셀에서 진행).")



--- [3단계: DTW 거리 행렬 계산] ---
3.2 클러스터링 대상 SKU 수: 117
3.5 DTW 거리 행렬 계산 완료.
    dist_matrix shape: (117, 117)
    dist_matrix[0:3, 0:3] 예시:
[[ 0.          5.43735229 11.31886586]
 [ 5.43735229  0.         13.35943765]
 [11.31886586 13.35943765  0.        ]]
3.6 대칭 여부: True
    음수 거리 존재?: False

✅ 3단계 완료:
 - sku_order: 거리행렬 인덱스와 SKU를 매핑할 리스트
 - dist_matrix: SKU 간 DTW 거리행렬 (shape = [N_SKU, N_SKU])
이제 이걸 가지고 k 후보별 클러스터링과 실루엣 스코어를 구할 수 있습니다 (다음 셀에서 진행).


In [6]:
# --- 4단계: k 후보별 클러스터링 (Agglomerative v2), 실루엣 스코어 계산, best_k 선택, cluster_map 생성 ---

import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score

print("\n--- [4단계: k 최적화 및 cluster_map 생성 - Agglomerative v2] ---")

# 전 단계 산출물 필요
needed_vars = ["dist_matrix", "sku_order", "lowdata_skus"]
for var_name in needed_vars:
    if var_name not in globals():
        raise RuntimeError(f"🚨 '{var_name}' 이(가) 정의되어 있지 않습니다. 이전 셀을 먼저 실행해야 합니다.")
    if globals()[var_name] is None:
        raise RuntimeError(f"🚨 '{var_name}' 이(가) None 입니다. 이전 셀에서 생성이 안 된 것 같습니다.")

# 후보 k 값들
k_candidates = [2, 3, 4, 5]

silhouette_results = []
label_results = {}

for k in k_candidates:
    if k >= len(sku_order):
        print(f"⚠ k={k} 는 SKU 수보다 많거나 같아서 스킵합니다.")
        continue

    # AgglomerativeClustering (신형 sklearn API)
    # - metric='precomputed' 로 DTW 거리행렬 사용
    # - linkage='average' 로 군집 간 거리 계산
    clusterer = AgglomerativeClustering(
        n_clusters=k,
        metric='precomputed',
        linkage='average'
    )

    # fit_predict에 거리행렬(dist_matrix)을 그대로 넣는다.
    labels = clusterer.fit_predict(dist_matrix)

    # silhouette_score 계산
    # silhouette_score(X, labels, metric='precomputed')
    # X는 거리행렬
    try:
        sil = silhouette_score(dist_matrix, labels, metric="precomputed")
    except Exception as e:
        print(f"⚠ k={k} 실루엣 계산 실패: {e}")
        sil = None

    silhouette_results.append({
        "k": k,
        "silhouette": sil
    })
    label_results[k] = labels

# 4.2 k별 실루엣 점수 출력
print("\n4.2 k 후보별 실루엣 점수:")
for res in silhouette_results:
    print(f"   k={res['k']}: silhouette={res['silhouette']}")

# 4.3 best_k 선택
valid_scores = [
    (res["k"], res["silhouette"])
    for res in silhouette_results
    if res["silhouette"] is not None
]

if len(valid_scores) == 0:
    raise RuntimeError("🚨 모든 k에 대해 실루엣 점수를 계산할 수 없습니다. 클러스터링이 실패했습니다.")

# 실루엣이 높은 k를 우선, 동률이면 더 작은 k를 선택
best_k, best_sil = sorted(
    valid_scores,
    key=lambda x: (-x[1], x[0])
)[0]

print(f"\n4.3 선택된 best_k = {best_k} (silhouette={best_sil})")

best_labels = label_results[best_k]

# 4.4 cluster_map_df 생성
cluster_map_df = pd.DataFrame({
    "Product_Number": sku_order,
    "cluster_id": best_labels
})

# 관측 부족 SKU(-1) 처리 (lowdata_skus는 2단계에서 수집)
if lowdata_skus:
    low_df = pd.DataFrame({
        "Product_Number": lowdata_skus,
        "cluster_id": [-1] * len(lowdata_skus)
    })
    cluster_map_df = pd.concat([cluster_map_df, low_df], ignore_index=True)

# 혹시라도 중복된 Product_Number가 생기면 마지막에 정리
cluster_map_df = cluster_map_df.drop_duplicates(subset=["Product_Number"]).reset_index(drop=True)

print("\n4.4 최종 cluster_map_df 예시 (상위 10개):")
print(cluster_map_df.head(10))

print(f"\n✅ 4단계 완료:")
print(f" - best_k = {best_k}, silhouette = {best_sil}")
print(f" - cluster_map_df shape: {cluster_map_df.shape}")
print("   이제 cluster_map_df를 train/test 전체에 merge해서 cluster_id라는 범주형 피처를 붙일 수 있습니다.")



--- [4단계: k 최적화 및 cluster_map 생성 - Agglomerative v2] ---

4.2 k 후보별 실루엣 점수:
   k=2: silhouette=0.1330377594277968
   k=3: silhouette=0.12959370372021664
   k=4: silhouette=0.16314839049493565
   k=5: silhouette=0.17174521232688753

4.3 선택된 best_k = 5 (silhouette=0.17174521232688753)

4.4 최종 cluster_map_df 예시 (상위 10개):
  Product_Number  cluster_id
0     Product_84           1
1     Product_85           1
2     Product_86           0
3     Product_87           0
4     Product_88           1
5     Product_89           0
6     Product_8a           0
7     Product_8b           1
8     Product_8c           1
9     Product_8d           1

✅ 4단계 완료:
 - best_k = 5, silhouette = 0.17174521232688753
 - cluster_map_df shape: (117, 2)
   이제 cluster_map_df를 train/test 전체에 merge해서 cluster_id라는 범주형 피처를 붙일 수 있습니다.


In [7]:
# --- 5단계: cluster_id를 train/test 전체에 merge하고 최종 CSV로 저장 ---

import pandas as pd
import os

print("\n--- [5단계: cluster_id 머지 & 최종 피처셋 생성] ---")

# 전 단계 산출물 점검
needed_vars = ["df_train", "df_test", "cluster_map_df"]
for var_name in needed_vars:
    if var_name not in globals() or globals()[var_name] is None:
        raise RuntimeError(f"🚨 '{var_name}' 이(가) 비어있습니다. 이전 셀들을 먼저 실행하세요.")

date_column = 'Date'
product_column = 'Product_Number'
target_column = 'T일 예정 수주량'

# 출력 경로 (최종 산출물)
output_final_csv_file = '../../data/dtw_clustering.csv'

# 5.1 train/test에 split 플래그 붙이기
train_labeled = df_train.copy()
train_labeled["__SPLIT__"] = "train"

test_labeled = df_test.copy()
test_labeled["__SPLIT__"] = "test"

# 5.2 cluster_id merge (왼쪽 기준: train/test, 오른쪽 기준: cluster_map_df)
train_merged = pd.merge(
    train_labeled,
    cluster_map_df,
    how="left",
    left_on=product_column,
    right_on="Product_Number"
)

test_merged = pd.merge(
    test_labeled,
    cluster_map_df,
    how="left",
    left_on=product_column,
    right_on="Product_Number"
)

# merge 후 중복된 Product_Number 컬럼 정리
# (left_on=Product_Number / right_on=Product_Number 이라 동일 이름이 생길 수 있어서)
if "Product_Number_y" in train_merged.columns:
    # 정리: Product_Number_x -> Product_Number, drop Product_Number_y
    train_merged.drop(columns=["Product_Number_y"], inplace=True, errors="ignore")
    train_merged.rename(columns={"Product_Number_x": "Product_Number"}, inplace=True)

if "Product_Number_y" in test_merged.columns:
    test_merged.drop(columns=["Product_Number_y"], inplace=True, errors="ignore")
    test_merged.rename(columns={"Product_Number_x": "Product_Number"}, inplace=True)

# 5.3 cluster_id 누락 체크
missing_train = train_merged["cluster_id"].isna().sum()
missing_test  = test_merged["cluster_id"].isna().sum()

if missing_train > 0 or missing_test > 0:
    print(f"⚠ 경고: cluster_id 매칭 안 된 행이 있습니다. train {missing_train}개, test {missing_test}개")
    # 이 상황이면 우리가 모르는 SKU가 test에 새로 등장했을 가능성.
    # 그 SKU는 클러스터링 당시(train 기준) 패턴이 없었을 수도 있음.
    # 이런 SKU는 cluster_id = -1으로 밀어넣는 게 안전하다.
    train_merged["cluster_id"] = train_merged["cluster_id"].fillna(-1)
    test_merged["cluster_id"]  = test_merged["cluster_id"].fillna(-1)

# 5.4 최종 df 결합
df_with_cluster = pd.concat([train_merged, test_merged], ignore_index=True)

# 5.5 sanity check 프린트
print("\n5.5 df_with_cluster 미리보기 (상위 5행):")
print(df_with_cluster[[product_column, date_column, target_column, "cluster_id", "__SPLIT__"]].head())

print("\n5.6 cluster_id 분포 (value_counts):")
print(df_with_cluster["cluster_id"].value_counts())

# 5.7 저장
# 이 df_with_cluster는 "공통 전처리 + (기존 피처들) + cluster_id"가 붙은 최종 피처셋이다.
# 다음 단계(LightGBM)에서 바로 쓸 수 있다.
os.makedirs(os.path.dirname(output_final_csv_file), exist_ok=True)
df_with_cluster.to_csv(output_final_csv_file, index=False, encoding='utf-8-sig')

print(f"\n✅ 5단계 완료: 최종 피처셋을 '{output_final_csv_file}'로 저장했습니다.")
print(f"   최종 shape: {df_with_cluster.shape}")
print("   이제 이 데이터를 가지고 LightGBM 실험 (baseline vs baseline+ts vs baseline+ts+cluster_id)으로 MAE 비교를 진행할 수 있습니다.")




--- [5단계: cluster_id 머지 & 최종 피처셋 생성] ---

5.5 df_with_cluster 미리보기 (상위 5행):
  Product_Number       Date  T일 예정 수주량  cluster_id __SPLIT__
0     Product_84 2022-01-26          0           1     train
1     Product_85 2022-01-26          0           1     train
2     Product_86 2022-01-26        616           0     train
3     Product_87 2022-01-26        104           0     train
4     Product_88 2022-01-26        187           1     train

5.6 cluster_id 분포 (value_counts):
cluster_id
1    4288
0    3674
2    1874
4     760
3      28
Name: count, dtype: int64

✅ 5단계 완료: 최종 피처셋을 '../../data/dtw_clustering.csv'로 저장했습니다.
   최종 shape: (10624, 21)
   이제 이 데이터를 가지고 LightGBM 실험 (baseline vs baseline+ts vs baseline+ts+cluster_id)으로 MAE 비교를 진행할 수 있습니다.


In [ ]:
# --- 5단계: cluster_id를 train/test 전체에 merge하고 최종 CSV로 저장 ---

import pandas as pd
import os

print("\n--- [5단계: cluster_id 머지 & 최종 피처셋 생성] ---")

# 전 단계 산출물 점검
needed_vars = ["df_train", "df_test", "cluster_map_df"]
for var_name in needed_vars:
    if var_name not in globals() or globals()[var_name] is None:
        raise RuntimeError(f"🚨 '{var_name}' 이(가) 비어있습니다. 이전 셀들을 먼저 실행하세요.")

date_column = 'Date'
product_column = 'Product_Number'
target_column = 'T일 예정 수주량'

# 최종 산출 파일 경로
output_final_csv_file = '../../data/dtw_clustering.csv'

# 5.1 train/test에 split 플래그 붙이기
train_labeled = df_train.copy()
train_labeled["__SPLIT__"] = "train"

test_labeled = df_test.copy()
test_labeled["__SPLIT__"] = "test"

# 5.2 cluster_id merge (왼쪽 = train/test, 오른쪽 = cluster_map_df)
train_merged = pd.merge(
    train_labeled,
    cluster_map_df,
    how="left",
    left_on=product_column,
    right_on="Product_Number"
)

test_merged = pd.merge(
    test_labeled,
    cluster_map_df,
    how="left",
    left_on=product_column,
    right_on="Product_Number"
)

# 병합 후 중복되는 Product_Number_* 정리
if "Product_Number_y" in train_merged.columns:
    train_merged.drop(columns=["Product_Number_y"], inplace=True, errors="ignore")
    train_merged.rename(columns={"Product_Number_x": "Product_Number"}, inplace=True)

if "Product_Number_y" in test_merged.columns:
    test_merged.drop(columns=["Product_Number_y"], inplace=True, errors="ignore")
    test_merged.rename(columns={"Product_Number_x": "Product_Number"}, inplace=True)

# 5.3 cluster_id 누락값 처리
missing_train = train_merged["cluster_id"].isna().sum()
missing_test  = test_merged["cluster_id"].isna().sum()

if missing_train > 0 or missing_test > 0:
    print(f"⚠ 경고: cluster_id 매칭 안 된 행이 있습니다. train {missing_train}개, test {missing_test}개")
    # train 기간엔 원래 다 있어야 정상인데, test에는 신규 SKU가 노출될 수 있음.
    # 그런 SKU는 cluster_id = -1로 태깅해서 '미분류/신규 리스크' 취급 가능.
    train_merged["cluster_id"] = train_merged["cluster_id"].fillna(-1)
    test_merged["cluster_id"]  = test_merged["cluster_id"].fillna(-1)

# 5.4 최종 df 결합
df_with_cluster = pd.concat([train_merged, test_merged], ignore_index=True)



# 5.5 sanity check 출력
print("\n5.5 df_with_cluster 미리보기 (상위 5행):")
print(df_with_cluster[[product_column, date_column, target_column, "cluster_id", "__SPLIT__"]].head())

print("\n5.6 cluster_id 분포 (value_counts):")
print(df_with_cluster["cluster_id"].value_counts())

# 5.7 저장
os.makedirs(os.path.dirname(output_final_csv_file), exist_ok=True)
df_with_cluster.to_csv(output_final_csv_file, index=False, encoding='utf-8-sig')

print(f"\n✅ 5단계 완료: 최종 피처셋을 '{output_final_csv_file}'로 저장했습니다.")
print(f"   최종 shape: {df_with_cluster.shape}")
print("   이제 이 데이터를 이용해서 LightGBM 실험 (baseline vs cluster_id 포함)으로 "
      "test MAE 비교를 진행할 수 있습니다.")



--- [5단계: cluster_id 머지 & 최종 피처셋 생성] ---

5.5 df_with_cluster 미리보기 (상위 5행):
   Product_Number       Date  T일 예정 수주량  cluster_id __SPLIT__
0               0 2022-01-26          0           1     train
1               1 2022-01-26          0           1     train
2               2 2022-01-26        616           0     train
3               3 2022-01-26        104           0     train
4               4 2022-01-26        187           1     train

5.6 cluster_id 분포 (value_counts):
cluster_id
1    4288
0    3674
2    1874
4     760
3      28
Name: count, dtype: int64

✅ 5단계 완료: 최종 피처셋을 '../../data/dtw_clustering.csv'로 저장했습니다.
   최종 shape: (10624, 21)
   이제 이 데이터를 이용해서 LightGBM 실험 (baseline vs cluster_id 포함)으로 test MAE 비교를 진행할 수 있습니다.
